# Python Fundamentals: Functions and Scoping

## Introduction

**Functions** are fundamental building blocks in Python (and most programming languages). They are named blocks of reusable code designed to perform a specific task. Using functions helps organize code, reduce repetition (DRY - Don't Repeat Yourself), improve readability, and make programs easier to debug and maintain.

**Scoping** refers to the rules that determine where in your program a particular variable can be accessed or modified. Understanding scope is crucial for avoiding bugs related to variable naming conflicts and unintended side effects.

This notebook covers:
*   Defining functions (`def`).
*   Function arguments (positional, keyword, default, `*args`, `**kwargs`).
*   Return values.
*   Docstrings and type hinting.
*   Python's scoping rules (LEGB).
*   The `global` and `nonlocal` keywords.
*   Closures.

## Real-World Analogies

*   **Functions:** Like recipes (take ingredients/arguments, perform steps, produce a dish/return value), tools in a toolbox (each does a specific job), or sub-routines in a manufacturing process.
*   **Scoping:** Like different levels of access or visibility in an organization (local office vs. regional HQ vs. global HQ) or nested boxes (a variable in an inner box might not be visible from outside).

## 1. Defining Functions (`def`)

**Explain:** Functions are defined using the `def` keyword, followed by the function name, parentheses `()` for parameters, and a colon `:`. The indented block below the `def` line contains the function's code.

**Syntax:**
```python
def function_name(parameter1, parameter2, ...):
    """Optional docstring explaining the function."""
    # Code block (function body)
    statement1
    statement2
    return value # Optional return statement
```

*   **Parameters:** Variables listed inside the parentheses in the function definition.
*   **Arguments:** Actual values passed to the function when it is called.
*   **Docstring:** A string literal (usually triple-quoted) right after the `def` line, explaining what the function does. Essential for documentation.
*   **`return` statement:** Exits the function and optionally sends back a value. If omitted, the function implicitly returns `None`.

In [16]:
# --- Demonstrate: Simple Function Definition ---
from typing import Union # For type hinting multiple possible types

def greet(name: str) -> str:
    """Returns a simple greeting message.

    Args:
        name: The name of the person to greet.

    Returns:
        A greeting string.
    """
    # This is the function body
    return f"Hello, {name}!"

# Calling the function
message = greet("Alice")
print(f"Function call: greet('Alice') -> {message}")

# Accessing the docstring
print(f"\nDocstring for greet():\n{greet.__doc__}")

# Function returning None implicitly
def print_message(msg: str) -> None: # Type hint indicates no return value
    """Prints a message to the console."""
    print(f"Message: {msg}")

return_value = print_message("Testing")
print(f"Return value of print_message(): {return_value}") # Output: None

Function call: greet('Alice') -> Hello, Alice!

Docstring for greet():
Returns a simple greeting message.

Args:
    name: The name of the person to greet.

Returns:
    A greeting string.

Message: Testing
Return value of print_message(): None


## 2. Function Arguments

Python offers flexible ways to define and pass arguments to functions.

### 2a. Positional and Keyword Arguments

*   **Positional Arguments:** Matched based on their order during the function call.
*   **Keyword Arguments:** Matched based on the parameter name specified during the function call. They can appear in any order *after* positional arguments.

In [17]:
def describe_pet(animal_type: str, pet_name: str) -> str:
    """Describes a pet.

    Args:
        animal_type: The type of animal.
        pet_name: The name of the pet.
    
    Returns:
        A descriptive string.
    """
    return f"I have a {animal_type} named {pet_name}."

# --- Calling with positional arguments ---
desc1 = describe_pet("hamster", "Harry")
print(f"Positional call: {desc1}")

# --- Calling with keyword arguments ---
desc2 = describe_pet(animal_type="dog", pet_name="Willie")
desc3 = describe_pet(pet_name="Goldie", animal_type="fish") # Order doesn't matter
print(f"Keyword call 1: {desc2}")
print(f"Keyword call 2: {desc3}")

# --- Calling with a mix (Positional must come first) ---
desc4 = describe_pet("cat", pet_name="Whiskers")
print(f"Mixed call: {desc4}")

# --- Incorrect calls ---
# describe_pet(pet_name="Buddy", "dog") # SyntaxError: positional argument follows keyword argument
# describe_pet("parrot") # TypeError: missing 1 required positional argument: 'pet_name'

Positional call: I have a hamster named Harry.
Keyword call 1: I have a dog named Willie.
Keyword call 2: I have a fish named Goldie.
Mixed call: I have a cat named Whiskers.


### 2b. Default Argument Values

You can provide default values for parameters. If an argument for that parameter isn't provided during the call, the default value is used.

**CRITICAL PITFALL:** Default values are evaluated *only once* when the function is defined, not each time it's called. Avoid using mutable objects (like lists or dictionaries) as default values, as modifications will persist across calls.

In [18]:
from typing import List, Optional

def power(base: float, exponent: float = 2.0) -> float:
    """Calculates base raised to the exponent power."""
    return base ** exponent

print(f"power(5): {power(5)}")             # Uses default exponent=2 -> 25.0
print(f"power(5, 3): {power(5, 3)}")       # Overrides default -> 125.0
print(f"power(base=3): {power(base=3)}")   # Uses default exponent -> 9.0
print(f"power(exponent=4, base=2): {power(exponent=4, base=2)}") # Overrides default -> 16.0

# --- Pitfall: Mutable Default Argument ---
def add_item_bad(item: str, target_list: List[str] = []) -> List[str]: # DANGER! Default is mutable list
    """Adds an item to a list (incorrectly uses mutable default)."""
    target_list.append(item)
    return target_list

list1 = add_item_bad("apple")
print(f"Mutable Default (Call 1): {list1}") # Output: ['apple']
list2 = add_item_bad("banana") # Modifies the SAME default list!
print(f"Mutable Default (Call 2): {list2}") # Output: ['apple', 'banana']

# --- Correct Way: Use None as default and create list inside ---
def add_item_good(item: str, target_list: Optional[List[str]] = None) -> List[str]:
    """Adds an item to a list correctly (uses None default)."""
    if target_list is None:
        target_list = [] # Create a new list each time if none provided
    target_list.append(item)
    return target_list

list3 = add_item_good("cherry")
print(f"\nCorrect Default (Call 1): {list3}") # Output: ['cherry']
list4 = add_item_good("date")
print(f"Correct Default (Call 2): {list4}") # Output: ['date']

power(5): 25.0
power(5, 3): 125
power(base=3): 9.0
power(exponent=4, base=2): 16
Mutable Default (Call 1): ['apple']
Mutable Default (Call 2): ['apple', 'banana']

Correct Default (Call 1): ['cherry']
Correct Default (Call 2): ['date']


### 2c. Arbitrary Positional Arguments (`*args`)

Using `*args` in the function definition allows it to accept any number of positional arguments. These arguments are collected into a tuple named `args` (the name `args` is convention, but the `*` is required).

In [19]:
from typing import Any, Tuple

def calculate_sum(*numbers: float) -> float:
    """Calculates the sum of an arbitrary number of arguments.
    
    Args:
        *numbers: A variable number of numeric arguments.
        
    Returns:
        The sum of the numbers.
    """
    print(f"  Inside calculate_sum, numbers received: {numbers} (Type: {type(numbers)})")
    total = 0.0
    for num in numbers:
        total += num
    return total

sum1 = calculate_sum(1, 2, 3)
print(f"calculate_sum(1, 2, 3) -> {sum1}\n")

sum2 = calculate_sum(10.5, 20.0, -5.5, 1.0)
print(f"calculate_sum(10.5, 20.0, -5.5, 1.0) -> {sum2}\n")

sum3 = calculate_sum() # No arguments
print(f"calculate_sum() -> {sum3}")

  Inside calculate_sum, numbers received: (1, 2, 3) (Type: <class 'tuple'>)
calculate_sum(1, 2, 3) -> 6.0

  Inside calculate_sum, numbers received: (10.5, 20.0, -5.5, 1.0) (Type: <class 'tuple'>)
calculate_sum(10.5, 20.0, -5.5, 1.0) -> 26.0

  Inside calculate_sum, numbers received: () (Type: <class 'tuple'>)
calculate_sum() -> 0.0


### 2d. Arbitrary Keyword Arguments (`**kwargs`)

Using `**kwargs` allows a function to accept any number of keyword arguments that haven't been defined explicitly as parameters. These arguments are collected into a dictionary named `kwargs` (again, `kwargs` is convention, `**` is required).

In [20]:
from typing import Dict, Any

def build_profile(first: str, last: str, **user_info: Any) -> Dict[str, Any]:
    """Builds a dictionary containing user information.

    Args:
        first: First name (required).
        last: Last name (required).
        **user_info: Arbitrary keyword arguments for additional info.

    Returns:
        A dictionary representing the user profile.
    """
    profile: Dict[str, Any] = {}
    profile['first_name'] = first
    profile['last_name'] = last
    print(f"  Inside build_profile, kwargs received: {user_info} (Type: {type(user_info)})")
    for key, value in user_info.items():
        profile[key] = value
    return profile

user1 = build_profile('Albert', 'Einstein', 
                        location='Princeton', 
                        field='Physics')
print(f"User Profile 1: {user1}\n")

user2 = build_profile('Marie', 'Curie', 
                        location='Paris', 
                        field='Chemistry', 
                        nobel_prizes=2)
print(f"User Profile 2: {user2}")

  Inside build_profile, kwargs received: {'location': 'Princeton', 'field': 'Physics'} (Type: <class 'dict'>)
User Profile 1: {'first_name': 'Albert', 'last_name': 'Einstein', 'location': 'Princeton', 'field': 'Physics'}

  Inside build_profile, kwargs received: {'location': 'Paris', 'field': 'Chemistry', 'nobel_prizes': 2} (Type: <class 'dict'>)
User Profile 2: {'first_name': 'Marie', 'last_name': 'Curie', 'location': 'Paris', 'field': 'Chemistry', 'nobel_prizes': 2}


### 2e. Combining Argument Types

Functions can combine all types of arguments. The required order in the function definition is:
1.  Standard positional arguments
2.  Default arguments
3.  `*args`
4.  Keyword-only arguments (optional, defined after `*` or `*args`)
5.  `**kwargs`

In [21]:
from typing import Any, Tuple, Dict

def kitchen_sink(name: str, count: int = 1, *items: str, separator: str = ', ', **attributes: Any):
    """Demonstrates various argument types."""
    print(f"--- Function Call --- ")
    print(f"Name (positional/keyword): {name}")
    print(f"Count (default/keyword): {count}")
    print(f"Items (*args tuple): {items}")
    print(f"Separator (keyword-only): {separator}")
    print(f"Attributes (**kwargs dict): {attributes}")
    print("--------------------\n")

# Various ways to call it
kitchen_sink("Box")
kitchen_sink("Bag", 3, "apple", "banana")
kitchen_sink("Shelf",5, "apple", "banana", "cherry",  color="brown", height=180)
kitchen_sink("Drawer", separator='; ', width=50, depth=40) # Note: items is empty tuple
kitchen_sink("Cupboard", 10, "plates", "cups", "glasses", separator=' | ', material="wood", locked=False)

--- Function Call --- 
Name (positional/keyword): Box
Count (default/keyword): 1
Items (*args tuple): ()
Separator (keyword-only): , 
Attributes (**kwargs dict): {}
--------------------

--- Function Call --- 
Name (positional/keyword): Bag
Count (default/keyword): 3
Items (*args tuple): ('apple', 'banana')
Separator (keyword-only): , 
Attributes (**kwargs dict): {}
--------------------

--- Function Call --- 
Name (positional/keyword): Shelf
Count (default/keyword): 5
Items (*args tuple): ('apple', 'banana', 'cherry')
Separator (keyword-only): , 
Attributes (**kwargs dict): {'color': 'brown', 'height': 180}
--------------------

--- Function Call --- 
Name (positional/keyword): Drawer
Count (default/keyword): 1
Items (*args tuple): ()
Separator (keyword-only): ; 
Attributes (**kwargs dict): {'width': 50, 'depth': 40}
--------------------

--- Function Call --- 
Name (positional/keyword): Cupboard
Count (default/keyword): 10
Items (*args tuple): ('plates', 'cups', 'glasses')
Separator 

## 3. Scoping (LEGB Rule)

**Explain:** Python determines the scope of a variable (where it can be accessed) using the **LEGB** rule, checked in this order:

1.  **L - Local:** Variables assigned within the current function (`def` or `lambda`).
2.  **E - Enclosing function locals:** Variables in the local scope of any enclosing functions (applies to nested functions).
3.  **G - Global:** Variables assigned at the top level of a module file, or explicitly declared `global` within a function.
4.  **B - Built-in:** Pre-assigned names in Python (e.g., `print`, `len`, `str`, `list`, exceptions like `ValueError`).

Python searches these scopes sequentially. The first occurrence of the variable name found is used.

In [22]:
# --- Demonstrate: LEGB Scope ---

# G - Global scope
global_var = "I am global"

def outer_function():
    # E - Enclosing function local scope
    enclosing_var = "I am enclosing"
    
    def inner_function():
        # L - Local scope
        local_var = "I am local"
        
        print(f"  Inside inner_function:")
        print(f"    Accessing local_var: {local_var}")         # L
        print(f"    Accessing enclosing_var: {enclosing_var}") # E
        print(f"    Accessing global_var: {global_var}")     # G
        print(f"    Accessing built_in (len): {len}")        # B
        
    print(f"Inside outer_function:")
    # print(f"  Trying to access local_var: {local_var}") # NameError: local_var not defined here
    print(f"  Accessing enclosing_var: {enclosing_var}")
    print(f"  Accessing global_var: {global_var}")
    inner_function() # Call the inner function

print("Outside all functions:")
print(f"Accessing global_var: {global_var}")
# print(f"Trying to access enclosing_var: {enclosing_var}") # NameError: enclosing_var not defined here
# print(f"Trying to access local_var: {local_var}") # NameError: local_var not defined here
outer_function() # Call the outer function

Outside all functions:
Accessing global_var: I am global
Inside outer_function:
  Accessing enclosing_var: I am enclosing
  Accessing global_var: I am global
  Inside inner_function:
    Accessing local_var: I am local
    Accessing enclosing_var: I am enclosing
    Accessing global_var: I am global
    Accessing built_in (len): <built-in function len>


## 4. Modifying Scopes (`global` and `nonlocal`)

**Explain:** By default, assigning to a variable inside a function creates a *local* variable. To modify variables in outer scopes, you need specific keywords:

*   **`global` keyword:** Declares that a variable inside a function refers to the *global* scope. Use this if you need to modify a global variable.
*   **`nonlocal` keyword:** Declares that a variable inside a *nested* function refers to a variable in an *enclosing* (but not global) scope. Use this to modify variables in the parent function(s).

**Caution:** Modifying global variables is generally discouraged as it can make code harder to understand and debug (creates hidden dependencies). Use `nonlocal` judiciously for specific patterns like closures.

In [23]:
# --- Demonstrate: global keyword ---
call_count = 0 # Global variable

def increment_counter():
    global call_count # Declare intent to modify the global variable
    call_count += 1
    print(f"  Inside increment_counter, global call_count is now: {call_count}")

print(f"Initial global call_count: {call_count}")
increment_counter()
increment_counter()
print(f"Final global call_count: {call_count}\n")

# --- Demonstrate: nonlocal keyword ---
def create_tracker():
    tracked_value = 100 # Enclosing scope variable
    
    def update_value(new_val: int):
        nonlocal tracked_value # Declare intent to modify the enclosing variable
        print(f"    Inside update_value: old tracked_value={tracked_value}")
        tracked_value = new_val
        print(f"    Inside update_value: new tracked_value={tracked_value}")
        
    def get_value() -> int:
        return tracked_value # Reads from enclosing scope (nonlocal not needed for reading)
    
    return update_value, get_value

updater, getter = create_tracker()
print(f"Initial tracked value via getter(): {getter()}")
updater(150)
print(f"Updated tracked value via getter(): {getter()}")

Initial global call_count: 0
  Inside increment_counter, global call_count is now: 1
  Inside increment_counter, global call_count is now: 2
Final global call_count: 2

Initial tracked value via getter(): 100
    Inside update_value: old tracked_value=100
    Inside update_value: new tracked_value=150
Updated tracked value via getter(): 150


## 5. Closures

**Explain:** A closure occurs when a nested function (defined inside another function) remembers and has access to variables from its enclosing scope, even after the outer function has finished executing.

The nested function, along with the captured variables from its environment, forms the closure.

**Use Cases:** Creating function factories (like `make_multiplier` or `create_tracker` above), data hiding/encapsulation (less common in Python than classes), implementing decorators.

In [24]:
from typing import Callable

def make_adder(n: int) -> Callable[[int], int]:
    """Returns a function that adds 'n' to its argument."""
    # 'n' is in the enclosing scope
    
    def adder(x: int) -> int:
        # This inner function 'adder' closes over 'n'
        # It remembers the value of 'n' from when make_adder was called
        return x + n 
    
    return adder # Return the inner function (the closure)

# Create specific adder functions
add_5 = make_adder(5)  # 'n' is 5 for this closure
add_100 = make_adder(100) # 'n' is 100 for this closure

print(f"Closure Example:")
# Even though make_adder has finished, add_5 remembers n=5
print(f"add_5(10): {add_5(10)}")   # Output: 15
print(f"add_5(20): {add_5(20)}")   # Output: 25

# add_100 remembers n=100
print(f"add_100(10): {add_100(10)}") # Output: 110

# You can inspect the closure's captured variables (for introspection)
if hasattr(add_5, '__closure__') and add_5.__closure__:
    print(f"Variables captured by add_5: {add_5.__closure__[0].cell_contents}") # Shows 5

Closure Example:
add_5(10): 15
add_5(20): 25
add_100(10): 110
Variables captured by add_5: 5


## Best Practices

*   **DRY Principle:** Use functions to avoid repeating code blocks.
*   **Single Responsibility:** Aim for functions that do one specific task well. This makes them easier to understand, test, and reuse.
*   **Meaningful Names:** Choose clear, descriptive names for functions and parameters.
*   **Docstrings:** Write informative docstrings (using standard formats like Google or NumPy style) to explain what the function does, its arguments, and what it returns.
*   **Type Hints:** Use type hints to improve code clarity, enable static analysis, and catch potential errors early.
*   **Avoid Mutable Defaults:** Use `None` as a default for mutable types and initialize them inside the function.
*   **Limit `global`/`nonlocal`:** Use `global` and `nonlocal` sparingly, as they can reduce code clarity and increase the chance of bugs. Often, passing values explicitly or using classes is a better approach.
*   **Keep Functions Short:** If a function becomes too long or complex, consider breaking it down into smaller helper functions.

## Common Pitfalls & Interview Questions

*   **Pitfall: Mutable Default Arguments:** Accidentally sharing state between function calls due to mutable defaults.
*   **Pitfall: Scope Confusion:** Modifying global or enclosing variables without using `global` or `nonlocal`, leading to `UnboundLocalError` or creating unintended local variables.
*   **Pitfall: Forgetting `return`:** Expecting a value back from a function that doesn't explicitly `return` one (it returns `None`).
*   **Pitfall: Argument Order:** Mixing positional and keyword arguments incorrectly.

*   **Interview Question:** "Explain the LEGB rule for variable scoping in Python."
    *   *Answer:* Local, Enclosing function locals, Global, Built-in. Describes the order Python searches for variable names.
*   **Interview Question:** "What is the difference between positional and keyword arguments?"
    *   *Answer:* Positional args are matched by order, keyword args by name. Keywords must follow positionals.
*   **Interview Question:** "What do `*args` and `**kwargs` do in a function definition?"
    *   *Answer:* `*args` collects extra positional arguments into a tuple. `**kwargs` collects extra keyword arguments into a dictionary.
*   **Interview Question:** "Why should you generally avoid using mutable types (like lists) as default argument values? What's the correct way to handle this?"
    *   *Answer:* Defaults are evaluated once at definition time. Mutable defaults are shared across calls. Use `None` as default and create the mutable object inside the function if needed.
*   **Interview Question:** "What is a closure in Python? Give an example."
    *   *Answer:* A nested function that remembers and accesses variables from its enclosing scope, even after the outer function returns. Example: a function factory like `make_adder`.
*   **Interview Question:** "When do you need to use the `global` or `nonlocal` keyword?"
    *   *Answer:* `global` to modify a variable in the global scope. `nonlocal` to modify a variable in an enclosing function's scope (but not global).

## 6. Challenge: Flexible Data Formatter

Write a function `format_data` that takes arbitrary key-value pairs (`**kwargs`) and formats them into a string.

1.  The function should accept any number of keyword arguments.
2.  It should also accept an optional keyword argument `prefix` (defaulting to an empty string) which should be added before each key-value pair string.
3.  It should also accept an optional keyword argument `separator` (defaulting to `", "`) to join the formatted key-value strings.
4.  The function should iterate through the `kwargs` (excluding `prefix` and `separator` if passed via `kwargs`) and create strings in the format `"key=value"`.
5.  Each formatted string should have the `prefix` prepended.
6.  Join these prefixed strings using the specified `separator`.
7.  Return the final combined string.

In [25]:
from typing import Any, Dict

def format_data(**kwargs: Any) -> str:
    """Formats key-value pairs into a string with prefix and separator.

    Args:
        **kwargs: Arbitrary key-value pairs. Can include optional keys 
                  'prefix' (str) and 'separator' (str) for formatting.

    Returns:
        A formatted string.
    """
    # Extract formatting args with defaults, remove them from kwargs if present
    prefix = kwargs.pop('prefix', '') # Default to empty string
    separator = kwargs.pop('separator', ', ') # Default to ', '
    
    formatted_parts: List[str] = []
    # kwargs now only contains the data key-value pairs
    for key, value in kwargs.items():
        formatted_parts.append(f"{prefix}{key}={value}")
        
    return separator.join(formatted_parts)

# --- Test the function ---
result1 = format_data(name="Test", value=100, status="Active")
print(f"Test 1 (defaults): '{result1}'")

result2 = format_data(user="admin", id=123, prefix="--", separator=' | ')
print(f"Test 2 (custom format): '{result2}'")

result3 = format_data(a=1, b=2, c=3, prefix='  -> ', separator='\n')
print(f"Test 3 (multiline format):\n'{result3}'")

result4 = format_data(prefix='Item: ', separator='; ')
print(f"Test 4 (no data args): '{result4}'")

Test 1 (defaults): 'name=Test, value=100, status=Active'
Test 2 (custom format): '--user=admin | --id=123'
Test 3 (multiline format):
'  -> a=1
  -> b=2
  -> c=3'
Test 4 (no data args): ''


## Quiz

1.  What is the primary purpose of using functions?
    a) To make code run faster.
    b) To store large amounts of data.
    c) To organize code, reduce repetition, and improve readability.
    d) To handle file input/output.

2.  In the LEGB rule, what does 'E' stand for?
    a) External
    b) Environmental
    c) Enclosing function locals
    d) Executable

3.  What will `my_func('a', 'b', c=3, d=4)` print if `def my_func(x, *args, **kwargs): print(args, kwargs)`?
    a) `('a', 'b') {'c': 3, 'd': 4}`
    b) `('b',) {'c': 3, 'd': 4}`
    c) `('a', 'b', 'c', 'd') {}`
    d) `() {'x': 'a', 'args': ('b',), 'c': 3, 'd': 4}`

4.  Why is `def add_nums(nums=[]): nums.append(1); print(nums)` considered bad practice?
    a) Function names cannot contain underscores.
    b) The default list `[]` is mutable and shared across calls.
    c) `append` is an inefficient list operation.
    d) Print statements should not be inside functions.

5.  When is the `nonlocal` keyword used?
    a) To modify a global variable from within any function.
    b) To modify a variable in an enclosing function's scope from a nested function.
    c) To declare a variable that exists only outside of functions.
    d) To access built-in functions.

*(Answers: 1-c, 2-c, 3-b, 4-b, 5-b)*

## Conclusion

Functions and scoping are core concepts in Python. Mastering function definition, understanding the different ways to handle arguments (`*args`, `**kwargs`, defaults), and grasping the LEGB scope rules (along with `global` and `nonlocal`) are essential for writing robust, maintainable, and bug-free Python applications. Closures provide a powerful mechanism for creating specialized functions and managing state.